In [8]:
from pyhealth.datasets import MIMIC3Dataset
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import Transformer, RNN
from pyhealth.tasks import mortality_prediction_mimic3_fn, drug_recommendation_mimic3_fn
from pyhealth.trainer import Trainer

dataset = MIMIC3Dataset(
    root='/data/physionet.org/files/mimiciii/1.4/',
    tables=["DIAGNOSES_ICD", "PROCEDURES_ICD", "PRESCRIPTIONS"],
    code_mapping={
        "ICD9CM": "CCSCM", 
        "ICD9PROC": "CCSPROC",
        "NDC": ("ATC", {"target_kwargs": {"level": 3}})
        },
    dev=False,
    refresh_cache=False
)


In [10]:
mimic3_ds = dataset.set_task(drug_recommendation_mimic3_fn)

train_dataset, val_dataset, test_dataset = split_by_patient(
    mimic3_ds, [0.8, 0.1, 0.1]
)
train_dataloader = get_dataloader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = get_dataloader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = get_dataloader(test_dataset, batch_size=32, shuffle=False)

Generating samples for drug_recommendation_mimic3_fn: 100%|██████████| 46520/46520 [00:02<00:00, 19741.61it/s]


In [17]:
model_w_pre = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
    pretrained_emb="LM",
    embedding_dim=768,
)

Loading pretrained embedding for conditions...
Loading pretrained embedding for procedures...


In [18]:
# STEP 4: define trainer
trainer = Trainer(model=model_w_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="pr_auc_samples",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(266, 1536, padding_idx=0)
    (procedures): Embedding(207, 1536, padding_idx=0)
  )
  (linear_layers): ModuleDict(
    (conditions): Linear(in_features=1536, out_features=768, bias=True)
    (procedures): Linear(in_features=1536, out_features=768, bias=True)
  )
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=768, out_features=768, bias=False)
              (1): Linear(in_features=768, out_features=768, bias=False)
              (2): Linear(in_features=768, out_features=768, bias=False)
            )
            (output_linear): Linear(in_features=768, out_features=768, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): Positionw

Epoch 0 / 10: 100%|██████████| 353/353 [00:03<00:00, 97.17it/s]

--- Train epoch-0, step-353 ---
loss: 0.3051



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 260.33it/s]


--- Eval epoch-0, step-353 ---
pr_auc_samples: 0.7006
loss: 0.2761
New best pr_auc_samples score (0.7006) at epoch-0, step-353



...

--- Train epoch-9, step-3530 ---
loss: 0.2710



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 247.03it/s]


--- Eval epoch-9, step-3530 ---
pr_auc_samples: 0.7140
loss: 0.2669
Loaded best model


Evaluation: 100%|██████████| 45/45 [00:00<00:00, 242.77it/s]


{'pr_auc_samples': 0.7158427348069376, 'loss': 0.2679338147242864}


In [13]:
model_no_pre = Transformer(
    dataset=mimic3_ds,
    feature_keys=["conditions", "procedures"],
    label_key="drugs",
    mode="multilabel",
)

In [16]:
# STEP 4: define trainer
trainer = Trainer(model=model_no_pre)
trainer.train(
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    epochs=10,
    monitor="pr_auc_samples",
)

# STEP 5: evaluate
print(trainer.evaluate(test_dataloader))

Transformer(
  (embeddings): ModuleDict(
    (conditions): Embedding(266, 128, padding_idx=0)
    (procedures): Embedding(207, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (transformer): ModuleDict(
    (conditions): TransformerLayer(
      (transformer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadedAttention(
            (linear_layers): ModuleList(
              (0): Linear(in_features=128, out_features=128, bias=False)
              (1): Linear(in_features=128, out_features=128, bias=False)
              (2): Linear(in_features=128, out_features=128, bias=False)
            )
            (output_linear): Linear(in_features=128, out_features=128, bias=False)
            (attention): Attention()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, 

Epoch 0 / 10: 100%|██████████| 353/353 [00:03<00:00, 96.92it/s]

--- Train epoch-0, step-353 ---
loss: 0.2501



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 239.97it/s]


--- Eval epoch-0, step-353 ---
pr_auc_samples: 0.7319
loss: 0.2561
New best pr_auc_samples score (0.7319) at epoch-0, step-353



...

--- Train epoch-9, step-3530 ---
loss: 0.2395



Evaluation: 100%|██████████| 45/45 [00:00<00:00, 177.23it/s]


--- Eval epoch-9, step-3530 ---
pr_auc_samples: 0.7295
loss: 0.2651
Loaded best model


Evaluation: 100%|██████████| 45/45 [00:00<00:00, 269.94it/s]


{'pr_auc_samples': 0.7230120046162672, 'loss': 0.2581981142361959}
